In [12]:
import json
import os

def get_best_and_worst_accuracy(folder_paths):
    all_runs = []
    
    for folder_path in folder_paths:
        if not os.path.exists(folder_path):
            print(f"Folder {folder_path} does not exist. Skipping.")
            continue

        for filename in os.listdir(folder_path):
            if filename.endswith('.json'):
                with open(os.path.join(folder_path, filename), 'r') as f:
                    data = json.load(f)
                    
                    try:
                        accuracy = data["test_run"]["scores"]["accuracy"]
                        variables = data["test_run"]["variables"]
                    except KeyError:
                        print(f"Could not find the required keys in {filename}. Skipping.")
                        continue
                    
                    all_runs.append({
                        "file": filename,
                        "accuracy": accuracy,
                        "variables": variables
                    })

    # Sort all_runs by accuracy
    sorted_runs = sorted(all_runs, key=lambda x: x['accuracy'], reverse=True)

    # Extract the best and worst runs
    best_run = sorted_runs[0] if sorted_runs else None
    worst_run = sorted_runs[-1] if sorted_runs else None

    # Print sorted runs
    for i, run in enumerate(sorted_runs):
        print(f"Run {i + 1}: {run['file']}")
        print(f"  Accuracy: {run['accuracy']}")
        print(f"  Variables: {run['variables']}")

    return {
        "best": best_run,
        "worst": worst_run
    }

# Specify the folder where .json files are located
folder_paths = ['results/full runs/training_iteration_4']

results = get_best_and_worst_accuracy(folder_paths)

if results['best'] and results['worst']:
    print(f"\nThe greatest accuracy is {results['best']['accuracy']}, found in {results['best']['file']}.")
    print(f"Variables for best run: {results['best']['variables']}")
    print(f"The lowest accuracy is {results['worst']['accuracy']}, found in {results['worst']['file']}.")
    print(f"Variables for worst run: {results['worst']['variables']}")
else:
    print("No valid runs found.")

Run 1: 2023-09-18_03-21-49_1.json
  Accuracy: 0.8239700374531835
  Variables: {'challenges_to_run_var': [], 'regex_type': 'specific', 'agent_explanation': 0, 'expert_prompt': 1, 'task_context': 1, 'scoring_type': 'trinary', 'reasoning_included': 1, 'few_shot_examples': 1, 'prompt_included': 0}
Run 2: 2023-09-18_05-47-08_1.json
  Accuracy: 0.8232209737827715
  Variables: {'challenges_to_run_var': [], 'regex_type': 'specific', 'agent_explanation': 0, 'expert_prompt': 1, 'task_context': 1, 'scoring_type': 'trinary', 'reasoning_included': 1, 'few_shot_examples': 0, 'prompt_included': 1}
Run 3: 2023-09-17_17-42-19_1.json
  Accuracy: 0.8209737827715355
  Variables: {'challenges_to_run_var': [], 'regex_type': 'specific', 'agent_explanation': 1, 'expert_prompt': 1, 'task_context': 1, 'scoring_type': 'trinary', 'reasoning_included': 1, 'few_shot_examples': 0, 'prompt_included': 0}
Run 4: 2023-09-18_08-58-01_1.json
  Accuracy: 0.8209737827715355
  Variables: {'challenges_to_run_var': [], 'regex_

In [1]:
import json
import os
from statistics import mean

sorted_metrics_list = []

def get_statistics_by_variables(folder_paths):
    # Initialize data structures to hold values for computing means
    var_metrics = {
        "agent_explanation": {},
        "expert_prompt": {},
        "task_context": {},
        "reasoning_included": {},
        "few_shot_examples": {},
        "prompt_included": {}
    }

    for folder_path in folder_paths:
        print(f"Checking folder {folder_path}...")
        if not os.path.exists(folder_path):
            print(f"Folder {folder_path} does not exist. Skipping.")
            continue

        for filename in os.listdir(folder_path):
            if filename.endswith('.json'):
                with open(os.path.join(folder_path, filename), 'r') as f:
                    data = json.load(f)
                    
                    try:
                        variables = data["test_run"]["variables"]
                        accuracy = data["test_run"]["scores"]["accuracy"]
                        precision = data["test_run"]["scores"]["precision"]
                        recall = data["test_run"]["scores"]["recall"]
                        counters = data["test_run"]["scores"]["counters"]
                    except KeyError:
                        print(f"Could not find the required keys in {filename}. Skipping.")
                        continue
                    
                    # Store the metrics according to the variable settings
                    for var, val in variables.items():
                        if var in var_metrics:
                            var_metrics[var].setdefault(val, {}).setdefault('accuracy', []).append(accuracy)
                            var_metrics[var].setdefault(val, {}).setdefault('precision', []).append(precision)
                            var_metrics[var].setdefault(val, {}).setdefault('recall', []).append(recall)
                            var_metrics[var].setdefault(val, {}).setdefault('counters', []).append(counters)

    # Compute means
    for var, values in var_metrics.items():
        for val, metrics in values.items():
            mean_accuracy = mean(metrics['accuracy']) if metrics['accuracy'] else None
            mean_precision = mean(metrics['precision']) if metrics['precision'] else None
            mean_recall = mean(metrics['recall']) if metrics['recall'] else None
            mean_counters = {key: mean([counter[key] for counter in metrics['counters']]) for key in ['TP', 'TN', 'FP', 'FN']}
            
            # Append metrics to the list instead of printing
            sorted_metrics_list.append({
                'var': var,
                'val': val,
                'mean_accuracy': mean_accuracy,
                'mean_precision': mean_precision,
                'mean_recall': mean_recall,
                'mean_counters': mean_counters
            })
    
    # Sort the list by mean_accuracy
    sorted_metrics_list.sort(key=lambda x: x['mean_accuracy'], reverse=True)

    # Print sorted metrics
    for metric in sorted_metrics_list:
        print(f"For {metric['var']} = {metric['val']}:")
        print(f"  Mean Accuracy: {round(metric['mean_accuracy']*100, 3)}%")
        print(f"  Mean Precision: {round(metric['mean_precision']*100, 3)}%")
        print(f"  Mean Recall: {round(metric['mean_recall']*100, 3)}%")
        print(f"  Mean Counters: {metric['mean_counters']}")


# Specify the folder where .json files are located
folder_paths = ['results/full runs/final_optimization_1', 'results/full runs/final_optimization_2']

get_statistics_by_variables(folder_paths)


Checking folder results/full runs/final_optimization_1...
Checking folder results/full runs/final_optimization_2...
For reasoning_included = 1:
  Mean Accuracy: 78.325%
  Mean Precision: 69.648%
  Mean Recall: 27.908%
  Mean Counters: {'TP': 92.93333333333334, 'TN': 952.7, 'FP': 49.3, 'FN': 240.06666666666666}
For expert_prompt = 1:
  Mean Accuracy: 76.791%
  Mean Precision: 61.045%
  Mean Recall: 20.906%
  Mean Counters: {'TP': 69.61538461538461, 'TN': 955.5384615384615, 'FP': 46.46153846153846, 'FN': 263.38461538461536}
For agent_explanation = 0:
  Mean Accuracy: 76.704%
  Mean Precision: 59.022%
  Mean Recall: 19.551%
  Mean Counters: {'TP': 65.10344827586206, 'TN': 958.8965517241379, 'FP': 43.10344827586207, 'FN': 267.8965517241379}
For prompt_included = 0:
  Mean Accuracy: 76.561%
  Mean Precision: 61.028%
  Mean Recall: 20.42%
  Mean Counters: {'TP': 68, 'TN': 954.0869565217391, 'FP': 47.91304347826087, 'FN': 265}
For task_context = 0:
  Mean Accuracy: 76.526%
  Mean Precision: 6